In [1]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import AutoTokenizer, AutoModel
from utils.MIND import MIND, MIND_news
from utils.Manager import Manager

logger = logging.getLogger(__name__)

ModuleNotFoundError: No module named 'pandas'

In [1]:
from keybert import KeyBERT

doc = """
         Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal). 
         A supervised learning algorithm analyzes the training data and produces an inferred function, 
         which can be used for mapping new examples. An optimal scenario will allow for the 
         algorithm to correctly determine the class labels for unseen instances. This requires 
         the learning algorithm to generalize from the training data to unseen situations in a 
         'reasonable' way (see inductive bias).
      """
kw_model = KeyBERT()

/data/workspace/Peitian/ns/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1, 1), stop_words='english')

In [4]:
keywords

[('supervised', 0.6676),
 ('labeled', 0.4896),
 ('learning', 0.4813),
 ('training', 0.4134),
 ('labels', 0.3947)]

In [5]:
# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")
t3 = AutoTokenizer.from_pretrained('google/bigbird-roberta-base', cache_dir=config.path + "bert_cache/")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [10]:
t3.convert_ids_to_tokens(t3("I love you", padding='max_length', max_length=10)['input_ids']), t3("I love you", padding='max_length', max_length=10)

(['[CLS]',
  '▁I',
  '▁love',
  '▁you',
  '[SEP]',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>'],
 {'input_ids': [65, 415, 1943, 446, 66, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]})

In [ ]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'
# config.reducer = 'first'

# config.signal_length = 10
# config.scale = 'large'
# config.impr_size = 1000
# config.mode = 'dev'
# config.fast = True
# config.news = 21563

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
file_directory = manager.path + "MIND/MIND{}_{}/".format(manager.scale, manager.mode)
# manager.gather_same_user_impr()

a = MIND(manager, file_directory)
# b = MIND_news(manager)

In [ ]:
# manager.construct_whole_dataset()

In [3]:
# behaviors = pickle.load(open('data/cache/bert/MINDlarge_dev/1000/behaviors.pkl','rb'))
# news = pickle.load(open('data/cache/bert/MINDdemo_train/news.pkl','rb'))
news = pickle.load(open('data/cache/bert/MINDdemo_train/bm25.pkl','rb'))

enc = news['encoded_news']

In [ ]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDlarge_test/bm25.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

index = 37405
news = t.convert_ids_to_tokens(a.encoded_news_original[index])
mask = a.attn_mask_original[index]
subword = a.subwords_original[index]

for i,j,k in zip(news,mask,subword):
    print(i,j,k)

In [ ]:
# bm25, entity, first
index = 1

# news = t.convert_ids_to_tokens(a[index]['his_encoded_index'][1])
# mask = a[index]['his_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = a[index]['his_subword_index'][1]

# for i,j,k in zip(news,mask,subword):
#     print(i,j,k)
res = a[index]
ids = res['cdd_id'][1]
news = t.convert_ids_to_tokens(res['cdd_encoded_index'][1])
mask = res['cdd_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
subword = res['cdd_subword_index'][1]
print(ids)
for i,j,k in zip(news,mask,subword):
    print(i,j,k)

In [ ]:
news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k,z in zip(news,mask,subword,dedup):
    print(i,j,k,z)